#### Imports

In [1]:
import os
import nltk
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
nltk.download('vader_lexicon')
from scipy.special import softmax
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, XLNetTokenizer, TFXLNetForSequenceClassification

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


## Filtering 3-Star Reviews

In [4]:
# Load the combined reviews data from the saved file
combined_reviews_df = pd.read_csv('processed_data/combined_reviews.csv', encoding='utf-8', engine='python')

# Filter for reviews with rating = 3 to perform sentiment analysis on the same
three_stars = combined_reviews_df[combined_reviews_df['rating'] == 3]

#print(filtered_df.head())
print(f"Total reviews with rating of 3: {len(three_stars)}")

Total reviews with rating of 3: 60703


## *is_recommended* column in the dataset

To ensure we would recommend products well-regarded by users, we explored the *is_recommended*  column in our reviews dataset and performed a series of checks to evaluate its reliability. As per the following:

In [12]:
# Count the number of 1s, 0s, and NAs in the 'is_recommended' column for 3-5 stars reviews
is_recommended_counts = combined_reviews_df['is_recommended'].value_counts(dropna=False)
is_recommended_counts

,count
is_recommended,
1.0,594237
NaN,107124
0.0,39029


We can already tell that there are a lot of missing values in the *is_recommended* column. Either way, before proceeding, we’ll check if any 5-star reviews are incorrectly marked as not recommended, as we would expect these to generally be recommended. This will help ensure data quality.

In [13]:
# Filter 5-star reviews
five_star_reviews = combined_reviews_df[combined_reviews_df['rating'] == 5]

# Check how many of the 5-star reviews are not recommended
not_recommended_5_stars = five_star_reviews[five_star_reviews['is_recommended'] == 0]

# Count the number of 5-star reviews that are not recommended
not_recommended_count = not_recommended_5_stars.shape[0]

print(f"Number of 5-star reviews marked as not recommended: {not_recommended_count}")

# Show a random review_text from the not recommended 5-star reviews, random seed set for reproducibility
random_example = not_recommended_5_stars['review_text'].sample(1, random_state=42).iloc[0]

print("\nRandom example of 5-star review marked as not recommended:")
print(random_example)

Number of 5-star reviews marked as not recommended: 610

Random example of 5-star review marked as not recommended:
This stuff is sooo good. Literally makes your lips feel so soft


Since there are 610 5-star reviews marked as "not recommended," and a random review test shows a very positive review from a happy user, this *is_recommended* column can't be trusted. Therefore, we'll rely on sentiment analysis to accurately identify products users truly recommend and feel good about.

# Sentiment Analysis on Product Reviews

## xlnet-base-cased-product-review-sentiment-analysis

Below we use [xlnet-base-cased-product-review-sentiment-analysis](https://huggingface.co/dipawidia/xlnet-base-cased-product-review-sentiment-analysis)  model from Hugging face to perform sentiment analysis.

In [14]:
# Initialize the tokenizer and model
tokenizer = XLNetTokenizer.from_pretrained("dipawidia/xlnet-base-cased-product-review-sentiment-analysis")
model = TFXLNetForSequenceClassification.from_pretrained("dipawidia/xlnet-base-cased-product-review-sentiment-analysis")

# Function to analyze sentiment for a batch of reviews
def get_sentiment_batch(reviews_batch):
    tokenize_text = tokenizer(reviews_batch.tolist(), padding=True, truncation=True, return_tensors='tf', max_length=512)
    preds = model.predict(dict(tokenize_text))['logits']
    class_preds = np.argmax(tf.keras.layers.Softmax()(preds), axis=-1)
    labels = ['Positive' if pred == 1 else 'Negative' for pred in class_preds]
    return labels

# Batch size for processing reviews
batch_size = 64

# Initialize the progress bar for batching
num_batches = len(three_stars) // batch_size + (1 if len(three_stars) % batch_size != 0 else 0)

# List to hold sentiment results
sentiment_results = []

# Process reviews in batches
for i in tqdm(range(0, len(three_stars), batch_size), total=num_batches, desc="Processing reviews", unit="batch"):
    batch = three_stars['review_text'][i:i + batch_size]  # Get the current batch of reviews
    batch_sentiments = get_sentiment_batch(batch)  # Get sentiments for the current batch
    sentiment_results.extend(batch_sentiments)  # Append results

# Assign the sentiment results back to the dataframe
three_stars['sentiment'] = sentiment_results

# Count positive and negative sentiments
sentiment_counts = three_stars['sentiment'].value_counts()
print(sentiment_counts)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

...

Processing reviews: 100%|█████████▉| 948/949 [1:17:47<00:07,  7.43s/batch]

1/1 [==============================] - 3s 3s/step


Processing reviews: 100%|██████████| 949/949 [1:17:51<00:00,  4.92s/batch]

sentiment
Negative    38999
Positive    21704
Name: count, dtype: int64



<ipython-input-14-71cf0fc2eb74>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  three_stars['sentiment'] = sentiment_results


In [20]:
# Filter the reviews with positive sentiment
positive_reviews_xlnet = three_stars[three_stars['sentiment'] == 'Positive']

# Display the count of positive sentiment reviews
print(f"Number of positive sentiment reviews: {len(positive_reviews_xlnet)}")

print("Positive Sentiment Examples:")
print(positive_reviews_xlnet[['review_text']].head())

Number of positive sentiment reviews: 21704
Positive Sentiment Examples:
                                           review_text
14   It’s super glossy and looks amazing but doesn’...
40   This product is marketed as a sleeping mask bu...
53   I think this is overpriced for what it is - li...
92   Meh, I mean it’s ok. I like the Fenty skin one...
112  I used to love this product. Because it feels ...




---



## cardiffnlp/twitter-xlm-roberta-base-sentiment

In [23]:
# Model path for the new model
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = TFAutoModelForSequenceClassification.from_pretrained(model_path)

# Function to analyze sentiment for a batch of reviews
def get_sentiment_batch(reviews_batch):
    # Tokenize the batch of reviews
    tokenize_text = tokenizer(reviews_batch.tolist(), padding=True, truncation=True, return_tensors='tf', max_length=512)

    # Get predictions from the model
    preds = model(**tokenize_text)['logits']

    # Apply softmax to get class probabilities
    scores = softmax(preds.numpy(), axis=-1)

    # Get the class with the highest probability (0 = negative, 2 = positive)
    class_preds = np.argmax(scores, axis=-1)

    # Assign sentiment labels based on the class prediction
    labels = ['Negative' if pred == 0 else 'Neutral' if pred == 1 else 'Positive' for pred in class_preds]

    return labels

# Batch size for processing reviews
batch_size = 64

# Ensure the reviews are valid strings and filter out NaN values
three_stars['review_text'] = three_stars['review_text'].fillna('').astype(str)

# Initialize the progress bar for batching
num_batches = len(three_stars) // batch_size + (1 if len(three_stars) % batch_size != 0 else 0)

# List to hold sentiment results
sentiment_results = []

# Process reviews in batches
for i in tqdm(range(0, len(three_stars), batch_size), total=num_batches, desc="Processing reviews", unit="batch"):
    batch = three_stars['review_text'][i:i + batch_size]  # Get the current batch of reviews
    batch_sentiments = get_sentiment_batch(batch)  # Get sentiments for the current batch
    sentiment_results.extend(batch_sentiments)  # Append results

# Assign the sentiment results back to the dataframe
three_stars['sentiment'] = sentiment_results

# Count positive, negative, and neutral sentiments
sentiment_counts = three_stars['sentiment'].value_counts()
print(sentiment_counts)

tf_model.h5:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

All the layers of TFXLMRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.
<ipython-input-23-32828c06af50>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  three_stars['review_text'] = three_stars['review_text'].fillna('').astype(str)
Processing reviews: 100%|██████████| 949/949 [22:42<00:00,  1.44s/batch]

sentiment
Negative    34045
Positive    21846
Neutral      4812
Name: count, dtype: int64



<ipython-input-23-32828c06af50>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  three_stars['sentiment'] = sentiment_results


In [24]:
# Filter the positive sentiment reviews
positive_reviews_cardiffnlp = three_stars[three_stars['sentiment'] == 'Positive']

# Display the count of positive sentiment reviews
print(f"Number of positive sentiment reviews: {len(positive_reviews_cardiffnlp)}")

print("Positive Sentiment Examples:")
print(positive_reviews_cardiffnlp[['review_text', 'product_id']].head())

Number of positive sentiment reviews: 21846
Positive Sentiment Examples:
                                          review_text product_id
11  Does the gummy bear mask smell absolutely deli...    P420652
30  ive tried my friend’s a few times and it does ...    P420652
40  This product is marketed as a sleeping mask bu...    P420652
53  I think this is overpriced for what it is - li...    P420652
80  I tried it first from the mini version Sephora...    P420652


## Intersect all positive reviews found

In [33]:
# Merge the two positive review datasets based on the 'Unnamed: 0' column
intersected_positive_reviews = pd.merge(
    positive_reviews_xlnet[['Unnamed: 0', 'review_text', 'product_id', 'author_id', 'review_title',
                            'product_name', 'brand_name', 'price_usd']],
    positive_reviews_cardiffnlp[['Unnamed: 0', 'product_id', 'author_id', 'review_title',
                                'product_name', 'brand_name', 'price_usd']],
    on='Unnamed: 0',
    how='inner'  # 'inner' ensures only matching rows are returned
)

# Rename columns to avoid conflicts (the '_x' and '_y' suffixes)
intersected_positive_reviews.rename(columns={
    'review_text_x': 'review_text',
    'product_id_x': 'product_id',
    'author_id_x': 'author_id',
    'review_title_x': 'review_title',
    'product_name_x': 'product_name',
    'brand_name_x': 'brand_name',
    'price_usd_x': 'price_usd'
}, inplace=True)

# Now drop the columns from the second dataframe that are not needed (with the '_y' suffix)
intersected_positive_reviews.drop(columns=[col for col in intersected_positive_reviews.columns if col.endswith('_y')], inplace=True)

# Select only desired columns
final_columns = ['Unnamed: 0', 'author_id', 'review_text', 'review_title', 'product_id', 'product_name', 'brand_name', 'price_usd']
intersected_positive_reviews = intersected_positive_reviews[final_columns]

# Display the count of intersected positive sentiment reviews
print(f"Number of intersected positive sentiment reviews: {len(intersected_positive_reviews)}")

# Check the first few rows of the final dataframe
print(intersected_positive_reviews.head())

# Save the intersected positive reviews to a CSV file
intersected_positive_reviews.to_csv('processed_data/3_star_positive_reviews.csv', index=False)

Number of intersected positive sentiment reviews: 14454
  Unnamed: 0    author_id                                        review_text  \
0         55  46853998374  This product is marketed as a sleeping mask bu...   
1         70   1660910909  I think this is overpriced for what it is - li...   
2        115  43851162127  Meh, I mean it’s ok. I like the Fenty skin one...   
3        137  33060342567  I used to love this product. Because it feels ...   
4        282   1552661644  It’s a great mask for sealing in moisture to t...   

                            review_title product_id  \
0            Wanted to love but couldnt.    P420652   
1                                    NaN    P420652   
2                                     ok    P420652   
3                      Not bad, not good    P420652   
4  Great when paired with other products    P420652   

                                        product_name brand_name  price_usd  
0  Lip Sleeping Mask Intense Hydration with Vitam...   